In [6]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

import os

start_path = '../../../data/functional/dict/'
# print(os.path.join(start_path))

# if not os.path.exists(start_path + 'user_to_book.json') or \
#         not os.path.exists(start_path + 'book_to_user.json') or \
#         not os.path.exists(start_path + 'user_book_to_rating.json') or \
#         not os.path.exists(start_path + 'user_book_to_rating_test.json'):
#     import data.dict.preprocess_book_data

with open(start_path + 'user_to_book_all.json', 'rb') as f:
    user_to_book_all = pickle.load(f)

with open(start_path + 'book_to_user_all.json', 'rb') as f:
    book_to_user_all = pickle.load(f)

with open(start_path + 'user_to_book.json', 'rb') as f:
    user_to_book = pickle.load(f)

with open(start_path + 'book_to_user.json', 'rb') as f:
    book_to_user = pickle.load(f)

with open(start_path + 'user_book_to_rating.json', 'rb') as f:
    user_book_to_rating = pickle.load(f)

with open(start_path + 'user_book_to_rating_test.json', 'rb') as f:
    user_book_to_rating_test = pickle.load(f)

In [14]:
N = np.max(list(book_to_user_all.keys())) + 1

m1 = np.max(list(book_to_user_all.keys()))
m2 = np.max([m for (u, m), r in user_book_to_rating_test.items()])
M = max(m1, m2) + 1

K = 25  # number of neighbors we'd like to consider
limit = 3  # number of books users must have in common

In [15]:
neighbors = {}
averages = {}
deviations = {}

In [16]:
def process_user_outer(user_id):
    global averages
    global deviations
    global neighbors
    global limit
    global K

    books_i = user_to_book[user_id]
    books_i_set = set(books_i)

    ratings_i = {book: user_book_to_rating[(user_id, book)] for book in books_i}
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = {book: (rating - avg_i) for book, rating in ratings_i.items()}

    averages[user_id] = avg_i
    deviations[user_id] = dev_i

    sl = SortedList()
    for j in user_to_book.keys():
        books_j = user_to_book[j]
        books_j_set = set(books_j)
        common_books = (books_i_set & books_j_set)
        if len(common_books) >= limit:
            ratings_j = {book: user_book_to_rating[(j, book)] for book in books_j}
            avg_j = np.mean(list(ratings_j.values()))
            dev_j = {book: (rating - avg_j) for book, rating in ratings_j.items()}

            numerator = sum(dev_i[m] * dev_j[m] for m in common_books)
            sigma_i = np.sqrt(sum(dev_i[m] * dev_i[m] for m in common_books))
            sigma_j = np.sqrt(sum(dev_j[m] * dev_j[m] for m in common_books))

            w_ij = numerator / (sigma_i * sigma_j)

            sl.add((-w_ij, j))
            if len(sl) > K:
                del sl[-1]

    neighbors[user_id] = sl

In [17]:
keys_len = len(user_to_book.keys())
index = 0

for i in user_to_book.keys():
    global index
    global keys_len
    index += 1
    process_user_outer(i)

    if index == 1000:
        progress_difference = 0
        print("process: {:.2f}%".format(i / keys_len * 100))

/var/folders/lr/_klwybj56q90nyw5lnq7mj0h0000gn/T/ipykernel_69413/2957874800.py:32: RuntimeWarning: invalid value encountered in double_scalars
  w_ij = numerator / (sigma_i * sigma_j)


process: 11.44%


In [18]:
def predict(i, m):
    numerator = 0
    denominator = 0
    for neg_w, j in neighbors[i]:
        try:
            numerator += -neg_w * deviations[j][m]
            denominator += abs(neg_w)
        except KeyError:
            pass
    if denominator == 0:
        prediction = averages[i]
    else:
        prediction = numerator / denominator + averages[i]
    prediction = min(10, prediction)
    prediction = max(1, prediction)
    return prediction

In [19]:
train_predictions = []
train_targets = []

for (i, b), target in user_book_to_rating.items():
    if i in user_to_book.keys() and b in book_to_user.keys():
        prediction = predict(i, b)

        train_predictions.append(prediction)
        train_targets.append(target)

print("Середнє значення: ", np.mean(train_predictions))

test_predictions = []
test_targets = []

for (i, b), target in user_book_to_rating_test.items():
    if i in user_to_book.keys() and b in book_to_user.keys():
        prediction = predict(i, b)

        test_predictions.append(prediction)
        test_targets.append(target)


def mse(p, t):
    p = np.array(p)
    t = np.array(t)
    return np.sum((p - t) ** 2) / len(p)


print('СКП для сету тренувань:', mse(train_predictions, train_targets))
print('СКП для тестового сету:', mse(test_predictions, test_targets))

Середнє значення:  7.899816372828574
СКП для сету тренувань: 0.6158051083007932
СКП для тестового сету: 3.3679676056804984


In [ ]:
df_info = pd.read_csv('../../../data/edited/books-info-edited.csv')

In [ ]:
book_to_user.keys()

sorted_books = {k: v for k, v in sorted(book_to_user.items(), key=lambda item: len(item[1]), reverse=True)}
# print(sorted_books)

with open('../../../data/shrink/ratings-book-translation.json', 'rb') as f:
    ratings_book_translation = pickle.load(f)

reversed_translation = {v: k for k, v in ratings_book_translation.items()}

sorter = list(sorted_books.keys())
sorter = [reversed_translation[i] for i in sorter]
sortedIndex = dict(zip(sorter, range(len(sorter))))

df_info["rank"] = df_info["bookId"].map(sortedIndex)
df_info.sort_values(["rank"])[df_info.title.str.contains("Potter")]
# df_info.sort_values(["rank"])[df_info.title.str.contains("Potter") == True]

In [261]:
from random import randrange

MY_ID = 300000

user_to_book[MY_ID] = [ratings_book_translation[i] for i in [81958, 234002, 47570]]

for item in user_to_book[MY_ID]:
    book_to_user[item].append(MY_ID)
    user_book_to_rating[(MY_ID, item)] = randrange(7, 10)

In [262]:
process_user_outer(MY_ID)

/var/folders/lr/_klwybj56q90nyw5lnq7mj0h0000gn/T/ipykernel_19189/2957874800.py:32: RuntimeWarning: invalid value encountered in double_scalars
  w_ij = numerator / (sigma_i * sigma_j)


In [264]:
predicted_books = {}
for item in sorted_books.keys():
    if item not in user_to_book[MY_ID]:
        predicted_books[item] = predict(MY_ID, item)


predicted_books_sorted = [reversed_translation[k] for k, v in sorted(predicted_books.items(), key=lambda item: item[1], reverse=True)][:100]
predicted_books_sorted_kv = {predicted_books_sorted[i]: i for i in range(len(predicted_books_sorted))}

predicted_df = df_info[df_info["bookId"].isin(predicted_books_sorted)].copy()
predicted_df["rated_rank"] = predicted_df.bookId.map(predicted_books_sorted_kv)

final_books = predicted_df.dropna().sort_values(["rated_rank"])
final_books

,isbn,title,author,year,publisher,image_s,image_m,image_l,bookId,rank,rated_rank
5256,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,177167,34.0,0
600,0345313860,"The Vampire Lestat (Vampire Chronicles, Book II)",ANNE RICE,1986,Ballantine Books,http://images.amazon.com/images/P/0345313860.0...,http://images.amazon.com/images/P/0345313860.0...,http://images.amazon.com/images/P/0345313860.0...,17527,37.0,1
3720,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,143842,43.0,2
1539,0345339703,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,166721,58.0,3
2735,0590353403,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,31657,62.0,4
...,...,...,...,...,...,...,...,...,...,...,...
7486,0380762595,Until Forever,Johanna Lindsey,1995,Avon,http://images.amazon.com/images/P/0380762595.0...,http://images.amazon.com/images/P/0380762595.0...,http://images.amazon.com/images/P/0380762595.0...,157624,1956.0,95
6427,0312950489,The Black Echo (Detective Harry Bosch Mysteries),Michael Connelly,1993,St. Martin's Press,http://images.amazon.com/images/P/0312950489.0...,http://images.amazon.com/images/P/0312950489.0...,http://images.amazon.com/images/P/0312950489.0...,76710,2757.0,96
6899,051511264X,Prime Witness,Steven Paul Martini,1994,Jove Books,http://images.amazon.com/images/P/051511264X.0...,http://images.amazon.com/images/P/051511264X.0...,http://images.amazon.com/images/P/051511264X.0...,54705,2887.0,97
13401,0671007610,Olivia (Logan),V.C. Andrews,1999,Pocket Books,http://images.amazon.com/images/P/0671007610.0...,http://images.amazon.com/images/P/0671007610.0...,http://images.amazon.com/images/P/0671007610.0...,111932,3289.0,98
